In [120]:
import pandas as pd
import shapefile as shp  # Requires the pyshp package
import matplotlib.pyplot as plt
import geopy.distance
from tqdm import tqdm
import numpy as np
import seaborn as sns

In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import geopandas as gpd
#import pysal as ps
#from pysal.contrib.viz import mapping as maps

In [101]:
import cv2
from PIL import Image
import glob
import numpy as np
from tqdm import tqdm

In [26]:
traffic_df = pd.read_csv('../data/processed/counties_and_unitary_authorities_traffic.csv')

In [27]:
traffic_df.head()

,Unnamed: 0,E06000001,E06000002,E06000004,E06000005,E06000006,E06000007,E06000008,E06000009,E06000010,...,E10000027,E10000028,E10000029,E10000030,E10000031,E10000032,E10000034,W06000006,W06000021,W06000022
0,01 January 2020,0,45061,351799,629342,46691,1812739,1447545,376914,319868,...,824675,70793,2716803,2305701,634755,416820,675655,214796,117072,23318
1,02 January 2020,0,72182,517262,1122638,54967,2836581,2158666,542817,456350,...,1251526,134233,4037491,3346911,891351,662507,1126469,286864,149969,34126
2,03 January 2020,0,6649,395919,1052879,19027,2951010,2211460,451953,413531,...,1293025,37146,4120981,3439365,896244,693547,1130052,216249,154403,32380
3,04 January 2020,0,0,314053,840480,0,2352197,1761284,377604,338002,...,1081974,23235,3380845,2868961,776569,572081,964965,157289,132003,27460
4,05 January 2020,0,38857,436519,875275,61706,2319385,1788273,443499,389598,...,1027883,101307,3390259,2893839,812506,530582,892966,258965,141156,27574


In [4]:
lsoas = gpd.read_file(shape_file_path)

In [5]:
lsoas.head()

,objectid,ctyua19cd,ctyua19nm,ctyua19nmw,bng_e,bng_n,long,lat,st_areasha,st_lengths,geometry
0,1,E06000001,Hartlepool,None,447157,531476,-1.27023,54.676159,9.844169e+07,65270.260128,"POLYGON ((447213.900 537036.104, 447228.798 53..."
1,2,E06000002,Middlesbrough,None,451141,516887,-1.21099,54.544670,5.455359e+07,41055.809886,"POLYGON ((448489.897 522071.798, 448592.597 52..."
2,3,E06000003,Redcar and Cleveland,None,464359,519597,-1.00611,54.567520,2.538909e+08,101191.876086,"POLYGON ((455834.103 528110.599, 455845.003 52..."
3,4,E06000004,Stockton-on-Tees,None,444937,518183,-1.30669,54.556911,2.097308e+08,108085.255484,"POLYGON ((444157.002 527956.304, 444165.898 52..."
4,5,E06000005,Darlington,None,428029,515648,-1.56835,54.535339,1.974757e+08,107206.401677,"POLYGON ((423496.602 524724.299, 423497.204 52..."


In [34]:
traffic_columns = traffic_df.columns[1:]
geo_ids = lsoas['ctyua19cd'].values

In [ ]:
for i in range(traffic_df.shape[0]):
    
    district_traffic = traffic_df.loc[i].iloc[1:]
    trf = []
    # Identify the Regions which have traffic information & generate a list of Traffic Flow values
    for j in range(geo_ids.shape[0]):
        try:
            trf.append(district_traffic[geo_ids[j]])
        except Exception:
            trf.append(0)
    
    # Add Traffic Information to geopandas dataframe
    lsoas['Traffic'] = pd.Series(trf)
    f, ax = plt.subplots(1,1,figsize=(10,10))
    ax = lsoas.plot(axes=ax, column='Traffic', legend=True, )
    ax.set_axis_off()
    ax.set_title(f'Traffic flow by District on {traffic_df.loc[i].iloc[0]}')
    
    image_name = str(i).rjust(5,'0')
    
    ax.text(s='Road Traffic Flow',x=850000.5,y=200000, rotation=90, fontsize = 20)
    plt.savefig(f"../data/external/traffic_maps/{image_name}.png", dpi = 200)

In [135]:
output_video_file = "timelapse.avi"
frame_rate = 10
width = height = 2000
fourcc = cv2.VideoWriter_fourcc(*'XVID')
videowriter = cv2.VideoWriter(output_video_file,fourcc, frame_rate, (width,height))
image_paths = glob.glob('../data/external/traffic_maps/*.png')

In [138]:
for path in tqdm(image_paths):
    frame = cv2.imread(path)
    videowriter.write(frame)


videowriter.release()

100%|████████████████████████████████████████████████████████████████████████████████| 121/121 [00:09<00:00, 13.04it/s]
